In [ ]:
#| hide
from BarcodeSeqKit.core import *

# BarcodeSeqKit

> BarcodeSeqKit is a library designed for extracting and processing barcoded sequences from next-generation sequencing data.

This file will become your README and also the index of your documentation.

## Developer Guide

If you are new to using `nbdev` here are some useful pointers to get you started.

### Install BarcodeSeqKit in Development mode

```sh
# make sure BarcodeSeqKit package is installed in development mode
$ pip install -e .

# make changes under nbs/ directory
# ...

# compile to have changes apply to BarcodeSeqKit
$ nbdev_prepare
```

## Usage

### Installation

Install latest from the GitHub [repository][repo]:

```sh
$ pip install git+https://github.com/mtinti/BarcodeSeqKit.git
```

or from [conda][conda]

```sh
$ conda install -c mtinti BarcodeSeqKit
```

or from [pypi][pypi]


```sh
$ pip install BarcodeSeqKit
```


[repo]: https://github.com/mtinti/BarcodeSeqKit
[docs]: https://mtinti.github.io/BarcodeSeqKit/
[pypi]: https://pypi.org/project/BarcodeSeqKit/
[conda]: https://anaconda.org/mtinti/BarcodeSeqKit

### Documentation

Documentation can be found hosted on this GitHub [repository][repo]'s [pages][docs]. Additionally you can find package manager specific guidelines on [conda][conda] and [pypi][pypi] respectively.

[repo]: https://github.com/mtinti/BarcodeSeqKit
[docs]: https://mtinti.github.io/BarcodeSeqKit/
[pypi]: https://pypi.org/project/BarcodeSeqKit/
[conda]: https://anaconda.org/mtinti/BarcodeSeqKit

## How to use

Fill me in please! Don't forget code examples:

In [ ]:
1+1

2